In [2]:
%pip install transformers==4.31.0
%pip install peft==0.4.0
%pip install accelerate==0.21.0
%pip install bitsandbytes==0.40.2
%pip install safetensors==0.3.3 # remove this?
%pip install tokenizers==0.13.3 # remove this?
%pip install datasets==2.14.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for bitsandbytes==0.40.2 from https://files.pythonhosted.org/packages/32/ea/75961538b08e4ed568057198717aabdebeaf6f398b7692420532e6861754/bitsandbytes-0.40.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 121.2 MB/s eta 0:00:00 0:00:01m
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages

In [3]:
import os
import argparse
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    default_data_collator,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
import torch

import bitsandbytes as bnb

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 아마존 세이지메이커 스튜디오에서 QLoRA로 LLaMA 7B 미세 조정

최근 Tim Dettmers 등이 발표한 "[QLoRA: Quantization-aware Low-Rank Adapter Tuning for Language Generation](https://arxiv.org/abs/2106.09685)"논문에서 소개된 방법을 활용할 것입니다. QLoRA는 성능 저하 없이 미세 조정 과정에서 대규모 언어 모델의 메모리 사용량을 줄이는 새로운 기법입니다. QLoRA의 작동 원리를 간단히 요약하면 다음과 같습니다.

* 사전 학습된 모델을 4비트로 양자화하고 고정합니다.
* 작은, 학습 가능한 어댑터 레이어를 추가합니다.(LoRA)
* 어댑터 레이어만 미세 조정하고, 고정된 양자화된 모델을 콘텍스트로 사용합니다.

### 하드웨어 요구 사항

우리는 다양한 모델 크기에 적합한 인스턴스 유형을 결정하기 위해 여러 실험을 수행했습니다. 다음 표는 실험 결과를 보여줍니다. 표에는 인스턴스 유형, 모델 크기, 콘텍스트 길이, 최대 배치 크기가 나와 있습니다.

| 모델        | 인스턴스 유형     | 최대 배치 크기 | 콘텍스트 길이 |
|--------------|-------------------|----------------|----------------|
| [LLama 7B]() | `(ml.)g5.4xlarge` | `3`            | `2048`         |
| [LLama 13B]() | `(ml.)g5.4xlarge` | `2`            | `2048`         |
| [LLama 70B]() | `(ml.)p4d.24xlarge` | `1++` (need to test more configs)            | `2048`         |


> `g5.4xlarge` 인스턴스 유형 대신 `g5.2xlarge`를 사용할 수 있지만, 이 경우 `merge_weights` 매개변수를 사용할 수 없습니다. LoRA 가중치를 모델 가중치에 병합하려면 모델이 메모리에 맞아야 하기 때문입니다. 하지만 학습 후 어댑터 가중치를 저장하고 [merge_adapter_weights.py](./scripts/merge_adapter_weights.py)를 활용해 병합할 수 있습니다.

_참고: 앞으로 이 목록을 확장할 계획입니다. 여러분의 설정을 자유롭게 공유 부탁드립니다!_

In [4]:
import argparse
parser = argparse.ArgumentParser()

# 모델 ID 및 데이터 세트 경로 인수 추가
parser.add_argument(
    "--model_id",
    type=str,
    default="NousResearch/Llama-2-7b-hf", # 게이트 아님, # TODO: 13b 시도하기
    help="Model id to use for training.",
)
parser.add_argument(
    "--dataset_path",
    type=str,
    default="lm_dataset",
    help="Path to dataset."
)
# parser.add_argument(
#     "--hf_token",
#     type=str,
#     default=HfFolder.get_token(),
#     help="Path to dataset."
# )
# 에포크, 배치 크기, 학습률, 시드를 위한 학습 하이퍼파라미터 추가
parser.add_argument(
    "--epochs",
    type=int,
    default=1,
    help="Number of epochs to train for."
)
parser.add_argument(
    "--per_device_train_batch_size",
    type=int,
    default=1,
    help="Batch size to use for training.",
)
parser.add_argument(
    "--lr",
    type=float,
    default=5e-5,
    help="Learning rate to use for training."
)
parser.add_argument(
    "--seed",
    type=int,
    default=42,
    help="Seed to use for training."
)
parser.add_argument(
    "--gradient_checkpointing",
    type=bool,
    default=True,
    help="Path to deepspeed config file.",
)
parser.add_argument(
    "--bf16",
    type=bool,
    # cuda 안쓰는 경우, default=False, 로 처리
    default=True if torch.cuda.get_device_capability()[0] >= 8 else False,
    #default=False,
    help="Whether to use bf16.",
)
parser.add_argument(
    "--merge_weights",
    type=bool,
    default=True,
    help="Whether to merge LoRA weights with base model.",
)
args, _ = parser.parse_known_args()

# if args.hf_token:
#     print(f"Logging into the Hugging Face Hub with token {args.hf_token[:10]}...")
#     login(token=args.hf_token)

In [5]:
# https://github.com/artidoro/qlora/blob/main/qlora.py에서 복사
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )


# https://github.com/artidoro/qlora/blob/main/qlora.py에서 복사
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)


def create_peft_model(model, gradient_checkpointing=True, bf16=True):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )
    from peft.tuners.lora import LoraLayer

    # INT4 모델 학습을 준비합니다.
    model = prepare_model_for_kbit_training(
        model, use_gradient_checkpointing=gradient_checkpointing
    )
    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # lora 대상 모듈 가져오기
    modules = find_all_linear_names(model)
    print(f"Found {len(modules)} modules to quantize: {modules}")

    peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        target_modules=modules,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )

    model = get_peft_model(model, peft_config)

    # 모델을 float32로 레이어 norms을 업캐스팅해 전처리합니다.
    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            if bf16:
                module = module.to(torch.bfloat16)
        if "norm" in name:
            module = module.to(torch.float32)
        if "lm_head" in name or "embed_tokens" in name:
            if hasattr(module, "weight"):
                if bf16 and module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)

    model.print_trainable_parameters()
    return model

## 데이터 세트 적재 및 준비

[InstructGPT 논문](https://arxiv.org/abs/2203.02155)에 설명된 여러 행동 범주(브레인스토밍, 분류, 폐쇄형 QA, 생성, 정보 추출, 개방형 QA, 요약)에 걸쳐 수천 명의 Databricks 직원들이 생성한 인스트럭션 수행 기록의 오픈 소스 데이터 세트인 [dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k)를 사용합니다.

```python
{
  "instruction": "World of Warcraft란 무엇인가?",
  "context": "",
  "response": "World of Warcraft는 대규모 온라인 멀티 플레이어 롤플레잉 게임입니다. 2004년에 Blizzard Entertainment에서 출시했습니다."
}
```

`samsum` 데이터 세트를 적재하기 위해 허깅 페이스 Datasets 라이브러리의 `load_dataset()` 메소드를 사용합니다.

In [6]:
# seed 설정
set_seed(args.seed)

from datasets import load_dataset
from random import randrange

# 허브에서 데이터 세트 로드
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
dataset = dataset.select(range(1000))

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

dataset size: 1000
{'instruction': 'Give me a list of some of the most popular song from 70s Japanese Pop singer, Mariya Takeuchi', 'context': '', 'response': '1. Plastic Love\n2. Stay with Me\n3. September\n4. Miracle Love\n5. Yume No Tsuzuki', 'category': 'brainstorming'}


모델을 인스트럭션 기반으로 미세 조정하려면 구조화된 예시를 인스트럭션으로 설명된 작업 모음으로 변환해야 합니다. 샘플을 받아 형식이 지정된 인스트럭션 문자열을 반환하는 `formatting_function`을 정의합니다.

In [7]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # 모든 부분을 하나로 결합합니다.
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt


In [8]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

### Instruction
What is the Research Collaboratory for Structural Bioinformatics Protein Data Bank (RCSB PDB)?

### Answer
The Research Collaboratory for Structural Bioinformatics Protein Data Bank (RCSB PDB) is a database that provides a wealth of information about the 3D structures of proteins, nucleic acids, and other macromolecules. The database contains experimentally determined atomic coordinates for a large number of macromolecules, which can be used to study their structures, functions, and interactions. The RCSB PDB is widely used in genomics research and drug discovery, as it provides a valuable resource for understanding the structural basis of many biological processes and for designing new drugs that target specific macromolecules.

In addition to the atomic coordinates, the RCSB PDB contains a wealth of additional information about each macromolecule, including experimental methods used for structure determination, citations to relevant scientific literature, and informat

In [9]:
from transformers import AutoTokenizer

#model_id = "meta-llama/Llama-2-13b-hf" # 조각화된 가중치, 게이트
model_id = "NousResearch/Llama-2-7b-hf" # 게이트 아님, TODO: 13b 시도하기
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [10]:
from random import randint
from itertools import chain
from functools import partial


# 각 샘플에 프롬프트를 추가하기 위한 템플릿 데이터 세트
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample


# 각 샘플에 프롬프트 템플릿 적용
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# 무작위 샘플 출력
print(dataset[randint(0, len(dataset))]["text"])

# 다음 배치에서 사용할 나머지를 저장할 빈 리스트
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # 정의된 전역 remainder 변수에 다음 배치에서 사용하기 위한 나머지 저장
    global remainder
    # 모든 텍스트를 연결하고 이전 배치의 나머지를 추가
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # 배치의 총 토큰 수 가져오기
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # 배치의 최대 청크 수 얻기
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # 최대 길이의 청크로 나누기
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # 다음 배치에서 사용할 나머지를 전역 변수에 추가
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # 레이블 준비
    result["labels"] = result["input_ids"].copy()
    return result


# 데이터 세트를 토큰화하고 청크로 나누기
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# 총 샘플 수 출력
print(f"Total number of samples: {len(lm_dataset)}")

### Instruction
Extract the owner of Lamborghini and a listing of the different types of Huracan cars that Lamborghini has produced for its Motorsport division.

### Context
Automobili Lamborghini S.p.A. (Italian pronunciation: [autoˈmɔːbili lamborˈɡiːni]) is an Italian manufacturer of luxury sports cars and SUVs based in Sant'Agata Bolognese. The company is owned by the Volkswagen Group through its subsidiary Audi.

Ferruccio Lamborghini (1916–1993), an Italian manufacturing magnate, founded Automobili Ferruccio Lamborghini S.p.A. in 1963 to compete with Ferrari. The company was noted for using a rear mid-engine, rear-wheel drive layout. Lamborghini grew rapidly during its first decade, but sales plunged in the wake of the 1973 worldwide financial downturn and the oil crisis. The firm's ownership changed three times after 1973, including a bankruptcy in 1978. American Chrysler Corporation took control of Lamborghini in 1987 and sold it to Malaysian investment group Mycom Setdco and In

In [11]:
# 위의 청크 처리는 행의 수를 줄입니다.
print(lm_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 105
})


In [12]:
# BNB 구성으로 허브에서 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    args.model_id,
    use_cache=False
    if args.gradient_checkpointing
    else True,  # 그래디언트 체크포인팅을 위해 필요합니다.
    device_map="auto",
    quantization_config=bnb_config,
)

# PEFT 구성 파일 생성
model = create_peft_model(
    model, gradient_checkpointing=args.gradient_checkpointing, bf16=args.bf16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Found 7 modules to quantize: ['v_proj', 'q_proj', 'down_proj', 'gate_proj', 'k_proj', 'up_proj', 'o_proj']
trainable params: 159,907,840 || all params: 3,660,320,768 || trainable%: 4.368683788535114


In [13]:
# 학습 인수 정의
output_dir = "./tmp/llama2_qlora"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=args.per_device_train_batch_size,
    bf16=args.bf16,  # BF16이 가능할 경우 사용
    learning_rate=args.lr,
    num_train_epochs=args.epochs,
    gradient_checkpointing=args.gradient_checkpointing,
    # 로깅 전략 설정
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
)

# Trainer 인스턴스 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    data_collator=default_data_collator,
)

# 학습 시작
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,1.756500
20,1.639400
30,1.568900
40,1.489800
50,1.596600
60,1.504300
70,1.452000
80,1.571800
90,1.647900
100,1.498300


TrainOutput(global_step=105, training_loss=1.5699456260317848, metrics={'train_runtime': 159.3204, 'train_samples_per_second': 0.659, 'train_steps_per_second': 0.659, 'total_flos': 4553577930424320.0, 'train_loss': 1.5699456260317848, 'epoch': 1.0})

In [14]:
adapter_save_dir = "./llama2_qlora_adapter"

tokenizer.save_pretrained(adapter_save_dir)

# 어댑터 가중치를 기본 모델과 병합한 후 저장
# INT4 모델 저장
trainer.model.save_pretrained(
    adapter_save_dir, safe_serialization=False
)

# 메모리 정리
del model
del trainer
torch.cuda.empty_cache()

In [15]:
from peft import AutoPeftModelForCausalLM

merged_save_dir = "./llama2_qlora_merged"

# 추론을 쉽게 하기 위해 토크나이저 저장
tokenizer.save_pretrained(merged_save_dir)

# FP16으로 PEFT 모델 로드
model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_save_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
)

# LoRA와 기본 모델 병합 후 저장
model = model.merge_and_unload()        
model.save_pretrained(
    merged_save_dir, safe_serialization=True, max_shard_size="2GB"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]